<h1> DattoRMM - Create Agent Install PowerShell Scripts <h1>

In [ ]:
# data import and file manipulation
import json

#data conditioning
import pandas as pd
import re
import datetime as dt

import requests
from requests.utils import CaseInsensitiveDict

In [ ]:
 # add current timestamp to filename for reference
current_time = (dt.datetime.utcnow().strftime('%Y_%m_%d_%H%M%S'))

# export folder will contain all csv exported DataFrames for Ticket Creation
export_folder = 'd:/exports/datto_install_scripts'

In [ ]:

import os
import hvac

def read_secret(mount_point, path):
    client = hvac.Client(url=os.environ.get("VAULT_ADDR"))
    resp = client.kv.v2.read_secret(mount_point=mount_point, path=f'/{path}')
    secret = resp['data']['data']

    return secret


In [ ]:
secrets = read_secret("api", "DATTO_RMM_CONFIG")

# Create Site Info DataFrames

## Create auth token

In [ ]:
# call token api url
token_uri = f'{secrets["base_uri"]}/auth/oauth/token'


# construct header
headers = CaseInsensitiveDict()
headers['Content-Type'] = 'application/x-www-form-urlencoded'

# construct req body
data = CaseInsensitiveDict()
data['grant_type'] = 'password'
data['username'] = secrets["api_key"]
data['password'] = secrets["api_secret"]

# request content response
resp = requests.post(token_uri, headers=headers, data=data, auth=('public-client', 'public'))
content = resp.content.decode('utf-8')
c_dict = json.loads(content)

access_token = c_dict['access_token']

## Pull Site Info

In [ ]:
## Create Devices DataFrame
# request content response
request_url = f'{secrets["base_uri"]}/api/v2/account/sites'

# construct header
headers = CaseInsensitiveDict()
headers['Authorization'] = f'Bearer {access_token}'
headers['Content-Type'] = 'application/json'

# construct req body
data = ''

print(f'Request URL: {request_url}')

resp = requests.get(request_url, headers=headers, data=data)
content = resp.content.decode('utf-8')
c_dict = json.loads(content)

df_sites = pd.DataFrame(c_dict['sites'])


df_site_info = df_sites[df_sites['name'] != 'Deleted Devices']

# Create PowerShell Template and Import DattoRMM Site Variables
## PowerShell Body Template

In [ ]:
def powershell_template(dictionary):

    datto_script_template = f"""

# -----------------------------------------------------------------------------------------------
# Title: DattoRMM Agent Install
# Author: Gabe McWilliams
# Purpose: Install DattoRMM Agent for Client: {dictionary['siteName']}
# Date of Creation: {dt.datetime.now().strftime("%Y/%m/%d")}
# Version 1.0
#
#  DattoRMM Deployment Script -- DO NOT RUN ON DEVICES WITH WORKING DATTORMM AGENTS! --
#
# ----------------------------------------------------------------------------------------------  #>

[Net.ServicePointManager]::SecurityProtocol =
[Net.SecurityProtocolType]::Tls12
If (Get-Service CagService -ErrorAction SilentlyContinue) \u007b
    Write-Output "Datto RMM Agent already installed on this device"
    Exit 0
    \u007d
Invoke-WebRequest -Uri {dictionary['agentDownloadUrl']} -OutFile c:\windows\\temp\DattoAgent.exe
Test-Path -Path C:\windows\\temp\DattoAgent.exe -PathType Leaf
c:\windows\\temp\DattoAgent.exe /qn


"""

    return datto_script_template

## Construct Download URL

In [ ]:
def agent_download_url(row):
    return f"https://concord.centrastage.net/csm/profile/downloadAgent/{row['uid']}"

## Create Powershell Installer Scripts

In [ ]:
dattormm_agent_installers = []
for index, row in df_site_info.iterrows():
    temp_dict = {}
    temp_dict['siteName'] = row['name']
    temp_dict['agentDownloadUrl'] = agent_download_url(row)
    dattormm_agent_installers.append(temp_dict)

In [ ]:
for e in dattormm_agent_installers:
    file_name = re.sub(r'(\s\-\s|\s+|\.\s?)','_',e['siteName'].lower())
    with open(f"{export_folder}/{file_name}_dattormm_agent_install.ps1", 'w+') as f:
        f.write(powershell_template(e))